## General Import

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler

import gc
import time
import seaborn as sns; sns.set()

import os 
# Disable warnings, set Matplotlib inline plotting and load Pandas package
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
#pd.options.display.mpl_style = 'default'
from datetime import datetime
import numpy as np
from datetime import datetime, timedelta
from pytz import timezone
from dateutil import tz
import geojson
import geopandas as gpd
from fiona.crs import from_epsg
import os, json
from shapely.geometry import shape, Point, Polygon, MultiPoint
%matplotlib inline
import matplotlib.pyplot as plt
from geopandas.tools import sjoin

import matplotlib.cm as cm

import folium
import shapely.geometry

from branca.colormap import  linear
import json
import branca.colormap as cm
import matplotlib.colors as colors
%matplotlib inline

import networkx as nx
import pickle

import osmnx as ox
ox.config(log_console=True, use_cache=True)
ox.__version__

import matplotlib.colors as mcolors


In [ ]:
!pip install spektral 

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from spektral.layers import GCNConv,  DiffusionConv, GATConv, GCSConv, GeneralConv, GlobalAvgPool, ChebConv
from spektral.utils import gcn_filter
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

In [ ]:
from numpy.random import seed

# Reproducability
def set_seed(seed=31415):
    
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    
set_seed(31415)

In [ ]:
path_graph = '../input/graph-results/graph_adj_bxl.pkl'

path_feat_flow = '../input/graph-results/FEATURES_flow_bxl.csv'
path_feat_vel = '../input/graph-results/FEATURES_vel_bxl.csv'

with open(path_graph,'rb') as f:
    graph, adj_matrix, edges, G = pickle.load(f)

adj_mx = nx.to_numpy_matrix(graph)

# flow
features_flow = pd.read_csv(path_feat_flow).iloc[:,1:].values
# vel
features_vel = pd.read_csv(path_feat_vel).iloc[:,1:].values

In [ ]:
# PARAMETERS

In [ ]:
inputs = 12
granularity = 2*2

## split train/val/test

In [ ]:
data_flow = features_flow[:, :-1]
data_vel = features_vel[:, :-1]

data = np.concatenate([data_flow, data_vel], axis=1)

In [ ]:
# Covariates

In [ ]:
week = 168*2*2
# week_test = 168*2

data_training = data[:-168*2*2*2, :]

list_training_validation = []

# Train/test split 0
data_tr_0, data_val_0 = data_training[:-672, :], data_training[-672:-448, :]
list_training_validation.append([data_tr_0, data_val_0])

# Train/test split 1
data_tr_1, data_val_1 = data_training[224:-448, :], data_training[-448:-224, :]
list_training_validation.append([data_tr_1, data_val_1])

# Train/test split 2
data_tr_2, data_val_2 = data_training[448:-224, :], data_training[-224:, :]
list_training_validation.append([data_tr_2, data_val_2])


datetime = features_flow[:, -1]

DATETIME = pd.DataFrame(datetime, columns=['Datetime'])
DATETIME['Datetime'] = pd.to_datetime(DATETIME['Datetime'])

DATETIME['minutes'] = DATETIME['Datetime'].dt.minute
DATETIME['hour'] = DATETIME['Datetime'].dt.hour

DATETIME['hour_x']=np.sin(DATETIME.hour*(2.*np.pi/23))
DATETIME['hour_y']=np.cos(DATETIME.hour*(2.*np.pi/23))

DATETIME['day'] = DATETIME['Datetime'].dt.day
DATETIME['DayOfWeek'] = DATETIME['Datetime'].dt.dayofweek

DATETIME['WorkingDays'] = DATETIME['DayOfWeek'].apply(lambda y: 2 if y < 5 else y)
DATETIME['WorkingDays'] = DATETIME['WorkingDays'].apply(lambda y: 1 if y == 5 else y)
DATETIME['WorkingDays'] = DATETIME['WorkingDays'].apply(lambda y: 0 if y == 6 else y)

DATETIME = DATETIME.drop(['Datetime','minutes','hour','day'], axis=1).values

# temporal features = 4
feat_time = 4


list_time = []


DATETIME_training = DATETIME[:-168*2*2*2, :]

# datetime Train/test split 0
time_tr_0, time_val_0 = DATETIME_training[:-672, :], DATETIME_training[-672:-448, :]
list_time.append([time_tr_0, time_val_0 ])

# datetime Train/test split 1
time_tr_1, time_val_1 = DATETIME_training[224:-448, :], DATETIME_training[-448:-224, :]
list_time.append([time_tr_1, time_val_1 ])

# datetime Train/test split 2
time_tr_2, time_val_2 = DATETIME_training[448:-224, :], DATETIME_training[-224:, :]
list_time.append([time_tr_2, time_val_2 ])

In [ ]:

l2_reg = 5e-4  # Regularization rate for l2

# Build model
class GCN_Net(Model):
    
    def __init__(self, state_size):
        
        super(GCN_Net, self).__init__()

        self.street = 4524
        
        self.head = 4
        
        self.state_size = 64
        
        self.state_size_gru = state_size


        # GCN 1st order approximation
        self.gcn_flow_0_enc = GCNConv(12, activation="relu", kernel_regularizer=l2(l2_reg), use_bias=False)
        self.gcn_flow_1_enc = GCNConv(12, activation="relu", kernel_regularizer=l2(l2_reg), use_bias=False)

        self.gcn_vel_0_enc = GCNConv(12, activation="relu", kernel_regularizer=l2(l2_reg), use_bias=False)
        self.gcn_vel_1_enc = GCNConv(12, activation="relu", kernel_regularizer=l2(l2_reg), use_bias=False)



###################

        self.lstm_flow_init = tf.compat.v1.keras.layers.CuDNNGRU(self.state_size_gru, #activation ='relu',
                                        kernel_initializer='glorot_uniform',
                                        recurrent_initializer='glorot_uniform',
                                        kernel_regularizer=regularizers.l2(0.001),
                                        bias_initializer='zeros', return_sequences = True, return_state =True)


        self.lstm_flow_fin = tf.compat.v1.keras.layers.CuDNNGRU(self.state_size_gru, #activation ='relu',
                                        kernel_initializer='glorot_uniform',
                                        recurrent_initializer='glorot_uniform',
                                        kernel_regularizer=regularizers.l2(0.001),
                                        bias_initializer='zeros')



        self.drop_flow = tf.keras.layers.Dropout(0.1)

        self.dense_flow_fin = tf.keras.layers.Dense(self.street*12,
                                           kernel_regularizer=regularizers.l2(0.001))
        
        

##########################


        self.lstm_vel_init = tf.compat.v1.keras.layers.CuDNNGRU(self.state_size_gru, #activation ='relu',
                                        kernel_initializer='glorot_uniform',
                                        recurrent_initializer='glorot_uniform',
                                        kernel_regularizer=regularizers.l2(0.001),
                                        bias_initializer='zeros', return_sequences = True, return_state =True)



        self.lstm_vel_fin = tf.compat.v1.keras.layers.CuDNNGRU(self.state_size_gru, #activation ='relu',
                                        kernel_initializer='glorot_uniform',
                                        recurrent_initializer='glorot_uniform',
                                        kernel_regularizer=regularizers.l2(0.001),
                                        bias_initializer='zeros')



        self.drop_vel = tf.keras.layers.Dropout(0.1)


        self.dense_vel_fin = tf.keras.layers.Dense(self.street*12,
                                           kernel_regularizer=regularizers.l2(0.001))
        
           
##################################

        self.attention_flow = tf.keras.layers.MultiHeadAttention(num_heads=self.head, key_dim=self.head)
        self.attention_vel = tf.keras.layers.MultiHeadAttention(num_heads=self.head, key_dim=self.head)


        self.dense_temporal_0 = tf.keras.layers.Dense(self.state_size_gru, activation ='relu') #tf.keras.layers.TimeDistributed()
        self.dense_temporal_1 = tf.keras.layers.Dense(self.state_size, activation ='relu')
        self.dense_temporal_2 = tf.keras.layers.Dense(self.state_size_gru)


        self.drop = tf.keras.layers.Dropout(0.1)   

        self.add = tf.keras.layers.Add()
        self.norm = tf.keras.layers.LayerNormalization()

        self.reshape = tf.keras.layers.Reshape([12, self.street])
        self.repeat = tf.keras.layers.RepeatVector(12)
        

    def call(self, flow_vel, a, past_cov, fut_cov):

        # sparse matrix
        sparse_a = tf.sparse.from_dense(a)
  
        # flow
        flow_0 = flow_vel[:, :, :-self.street]  
        # velocity
        vel_0  = flow_vel[:, :, -self.street:]
        

        # shape for gcn
        flow_sh = tf.reshape(flow_0 , [flow_0.shape[0], flow_0 .shape[2], flow_0.shape[1]])
        vel_sh = tf.reshape(vel_0 , [vel_0.shape[0], vel_0.shape[2], vel_0.shape[1]])

        # two gcn on flow and vel
        gcn_flow = self.gcn_flow_0_enc([flow_sh, sparse_a])
        gcn_flow = self.gcn_flow_1_enc([gcn_flow, sparse_a])

        gcn_vel = self.gcn_vel_0_enc([vel_sh, sparse_a])
        gcn_vel = self.gcn_vel_1_enc([gcn_vel, sparse_a])

        # shape for lstm
        flow_sh = tf.reshape(gcn_flow, [gcn_flow.shape[0], gcn_flow.shape[2], gcn_flow.shape[1]])
        vel_sh = tf.reshape(gcn_vel, [gcn_vel.shape[0], gcn_vel.shape[2], gcn_vel.shape[1]])

        # two lstm models
        flow_init, h_flow = self.lstm_flow_init(flow_sh)
        vel_init, h_vel = self.lstm_vel_init(vel_sh)

        # merge layer
        # output
        flow_vel_i = tf.concat([flow_init, vel_init], axis=2) # 

        flow_vel = self.dense_temporal_0(flow_vel_i)
        flow_vel = self.dense_temporal_1(flow_vel)
        flow_vel_f = self.dense_temporal_2(flow_vel)

        # multi-head attention
        # # Q: flow & vel
        # # K: flow
        att_flow, weight_flow = self.attention_flow(flow_vel_f, flow_init,
                               return_attention_scores=True)


        # # Q: vel & vel
        # # K: vel
        att_vel, weight_vel = self.attention_vel(flow_vel_f, vel_init,
                               return_attention_scores=True)

# -----

        # concatenate covariates
        # output flow_vel - covariates
        fut_cov = tf.cast(fut_cov, dtype=tf.float32)
        concat_flow = tf.concat([att_flow, fut_cov], axis=2)
        concat_vel = tf.concat([att_vel, fut_cov], axis=2)
  
        # two models for flow and speed respectively

        # flow
        flow_final = self.lstm_flow_fin(concat_flow , initial_state = [flow_vel_f[:,-1,:]]) # , initial_state = [flow_vel_f]) #  flow_vel_f, h_flow flow_vel_f[:,-1,:]
        flow = self.drop_flow(flow_final)
        flow = self.dense_flow_fin(flow)
        flow = self.reshape(flow)


        # velocity
        vel_final = self.lstm_vel_fin(concat_vel, initial_state = [flow_vel_f[:,-1,:]]) # , initial_state = [flow_vel_f]) 
        vel = self.drop_vel(vel_final)
        vel = self.dense_vel_fin(vel)
        vel = self.reshape(vel)


        # concatenate two finals results
        final = tf.concat([flow, vel], axis=-1)

        return final



loss_fn = tf.keras.losses.MeanAbsoluteError()

In [ ]:
adj_matrix = gcn_filter(adj_matrix, symmetric=True)

In [ ]:
# del data_tr, data_te, data_flow, data_vel, data, scaled_tr, scaled_te
# gc.collect()

In [ ]:
def prepare_data_DL(INPUT, FEAT, BATCH):
    
    dataset = FEAT.reshape(FEAT.shape[0], FEAT.shape[1]) 
    dataset = tf.data.Dataset.from_tensor_slices(dataset)
    
    inputs = dataset.window(INPUT,  shift=1,  stride=1,  drop_remainder=True)
    inputs = inputs.flat_map(lambda window: window.batch(INPUT))

    targets = dataset.window(INPUT, shift=1,  stride=1,  drop_remainder=True).skip(INPUT)
    targets = targets.flat_map(lambda window: window.batch(INPUT))

    dataset = tf.data.Dataset.zip((inputs, targets))
    dataset = dataset.batch(BATCH).prefetch(tf.data.experimental.AUTOTUNE)

    return dataset

In [ ]:
import time
import gc

start = time.time()

list_results = []

state_size_lst = [50, 150, 250]

epochs = 10

for state_size in state_size_lst:

    print('')
    print('state_size: '+str(state_size))
    print('EPOCHS: '+str(epochs))
    print('')
        
    list_avg_loss_split = []
    
    timeseriesplit = zip(list_training_validation, list_time)
    
    optimizer = Adam(lr=0.001)
    
    # Training function
    @tf.function
    def train_on_batch(inputs, target, past_cov, cov):
        loss = 0
        with tf.GradientTape() as tape:
            predictions = model(inputs, adj_matrix, past_cov, cov,  training=True)
            loss = loss_fn(target, predictions)
        variables = model.trainable_variables 
        gradients = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(gradients, variables))
        return loss
    
    # Create model
    model = GCN_Net(state_size)        
    
    for data, time in timeseriesplit:
        
        data_tr, data_val = data[0], data[1]
        time_tr, time_val = time[0], time[1]

        scaler = MinMaxScaler(feature_range=(0, 1))
        scaler_cov = MinMaxScaler(feature_range=(0, 1))

        # fit and transform
        scaled_tr = scaler.fit_transform(data_tr)
        # transform
        scaled_val= scaler.transform(data_val)

        # fit and transform
        scaled_tr_cov = scaler_cov.fit_transform(time_tr)
        # transform
        scaled_val_cov = scaler_cov.transform(time_val)
        
        batch_train = 64 
        batch_val = 64

        # features
        loader_tr = prepare_data_DL(12, scaled_tr, batch_train)
        loader_val = prepare_data_DL(12, scaled_val, batch_val)

        # covariates
        loader_tr_cov = prepare_data_DL(12, scaled_tr_cov, batch_train)
        loader_val_cov = prepare_data_DL(12, scaled_val_cov, batch_val)

        # Keep results for plotting
        train_loss_results = []
        val_loss_results = []

        samples_cov = list(loader_tr_cov)
        samples_cov_val = list(loader_val_cov)

        for epoch in range(epochs):

            ## training
            step = 0
            epoch_loss_avg = tf.keras.metrics.Mean()
            
            for batch in loader_tr:

                cov = samples_cov[step]
                past_cov = cov[0]
                fut_cov = cov[1]

                # Training step
                inputs, target = batch
                loss = train_on_batch(inputs, target, past_cov, fut_cov)
                # Track progress
                epoch_loss_avg.update_state(loss)
                step += 1

        ## validation 
        step_val = 0
        epoch_loss_avg_val = tf.keras.metrics.Mean()

        for batch in loader_val:

            cov = samples_cov_val[step_val]
            past_cov = cov[0]
            fut_cov = cov[1]

            # Validation step
            inputs, targ = batch
            pred = model(inputs, adj_matrix, past_cov, fut_cov,  training=False)
            loss = loss_fn(targ, pred)
            epoch_loss_avg_val.update_state(loss)
            step_val += 1

        list_avg_loss_split.append(epoch_loss_avg_val.result())
        print('loss per split: '+str(epoch_loss_avg_val.result()))

        del inputs, target,  past_cov, fut_cov
        gc.collect()
    
    print('')
    print('average loss per 3 split: '+str(np.mean(list_avg_loss_split)))
    list_results.append(np.mean(list_avg_loss_split))
    epochs = epochs + 10
    
     

In [ ]:
list_results_array = [np.array(x) for x in list_results]

In [ ]:
list_results_avg = [np.mean(x) for x in list_results_array]

In [ ]:
import pickle

# Saving the objects:
with open('results_GRU_bxl.pkl', 'wb') as f: 
    pickle.dump([list_results_array, list_results_avg], f)